In [17]:
import requests
import os
from bs4 import BeautifulSoup
import time
import re
import json

def create_soup(link):
    req = requests.get(link)
    if "Compare/GetCompareMerchant?" in link:
        json_data = json.loads(req.text)
        soup = BeautifulSoup(json_data['data'], 'html.parser')
        return soup
    soup = BeautifulSoup(req.text, 'html.parser')
    return soup 

def convert_id_to_link(item_id):
    return f'https://websosanh.vn/Compare/GetCompareMerchant?rootProductId={item_id}&regionId=0&pageIndex=1&sortType=1&pageSize=10'

def get_all_api_root(origin_link):
    req = requests.get(origin_link)
    json_data = json.loads(req.text)
    if int(json_data['totalRecords']) <= 10 and json_data['pagination'] == '':
        return [origin_link]
    soup = BeautifulSoup(json_data['pagination'], 'html.parser')
    last_page = find_last_page_of_result(soup)
    origin_link = origin_link.split('pageIndex=1')
    result = []
    for i in range(1, last_page+1):
        link_i_th = "{}pageIndex={}{}".format(origin_link[0], i, origin_link[1]) #https://websosanh.vn/s/?pi=i.htm
        result.append(link_i_th)
    return result

def get_children_item(link, title):
    soup = create_soup(link)
    result = []
    list_items = soup.find_all('li', class_='compare-item')
    for item in list_items:
        website = item.find('span', class_='compare-name')
        VAT_price = item.find('div', class_='compare-product-vat')
        item_price = item.find('div', class_='compare-product-price')
        if website and VAT_price and item_price:
            website = website.text
            VAT_price = ''.join(re.findall('[0-9]', VAT_price.text))
            if VAT_price:
                result.append({
                'title': title,
                'price': VAT_price,
                'website': website,
                'VAT': 1
                })
                continue
            item_price = ''.join(re.findall('[0-9]', item_price.text))
            result.append({
                'title': title,
                'price': item_price,
                'website': website,
                'VAT': 1 
            })
    return result

def get_children_items(list_links, title):
    result = []
    for link in list_links:
        result.extend(get_children_item(link, title))
    return result

def convert_string_to_link(search_key_word):
    search_key_word = ''.join([word for word in search_key_word if word.isalnum() or word == ' '])
    search_key_word = search_key_word.replace(' ','+')
    return f"https://websosanh.vn/s/{search_key_word}?pi=1.htm"

def find_last_page_of_result(soup):
    is_element = soup.find('a', class_ = 'last')
    if is_element:
        last_page = is_element['data-page']
        if last_page.isnumeric():
            return int(last_page)
    return 0

def get_data(link):
    soup = create_soup(link)
    result = []
    list_item_elements = soup.find_all('li', class_='product-item')
    for item_element in list_item_elements:
        title_element = item_element.find('h3')
        price_element = item_element.find('span', class_='product-price')
        link_element =  item_element.find('a')
        website_element = item_element.find('span',class_='product-store-logo')
        if title_element and price_element:
            title = title_element.text
            price = price_element.text
            price = ''.join(re.findall('[0-9]', price))
            is_item_stores = item_element.find('span', class_='product-store row-full')
            if is_item_stores:
                item_id = link_element['data-product-id']
                link = convert_id_to_link(item_id)     
                list_link = get_all_api_root(link)
                list_children_items = get_children_items(list_link, title)
                result.extend(list_children_items)
                print(list_children_items)
            elif website_element:
                store_element = website_element.find('img', class_='lazyload')
                website = store_element['alt']
                result.append({
                    'title': title,
                    'price': price,
                    'website': website,
                    'VAT': 0
                    })
    return result

def get_links(orgin_link):
    soup = create_soup(orgin_link)
    last_page = find_last_page_of_result(soup)
    orgin_link = orgin_link.split('?pi=1')
    result = []
    for i in range(1, last_page+1):
        link_i_th = "{}?pi={}{}".format(orgin_link[0], i, orgin_link[1]) #https://websosanh.vn/s/?pi=i.htm
        result.append(link_i_th)
    return result

def get_links_website(orgin_link):
    soup = create_soup(orgin_link)
    last_page = find_last_page_of_result(soup)
    orgin_link = orgin_link.split('?pi=1')
    result = []
    for i in range(1, last_page+1):
        link_i_th = "{}?pi={}{}".format(orgin_link[0], i, orgin_link[1]) #https://websosanh.vn/s/?pi=i.htm
        result.append(link_i_th)
    return result

def get_all_data(list_link):
    result = []
    if not list_link: print('No result, Can\'t get element')
    for link in list_link[:3]:
        data = get_data(link)
        result.extend(data)
    return result

def main(search_key_word):
    link = convert_string_to_link(search_key_word)
    list_link = get_links(link)
    data_crawl = get_all_data(list_link)
    data_input_file = {search_key_word: data_crawl}
#     data = []
#     if os.path.exists('data.json'):
#         _f = open('data.json', 'r', encoding='utf-8')
#         data_in_file = json.loads(_f.read())
#         data = data_in_file['data']
#         _f.close()
#     data.append(data_input_file)
#     data_in_file = {'data':data}

#     _f = open('data.json', 'w', encoding='utf-8')
#     json.dump(data_in_file, _f, indent=4, ensure_ascii=False)
#     _f.close()
a = main("điều hòa")

[{'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'khotongdienmay.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'dienmaythudo.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'dienmaymienbac.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'dienmaythudo24h.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'dienmayxuanminh.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'dienmayonline24h.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1800000', 'website': 'dientu365.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1

Viet Nam Work


In [18]:
a = [{'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'khotongdienmay.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'dienmaythudo.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'dienmaymienbac.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'dienmaythudo24h.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'dienmayxuanminh.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1790000', 'website': 'dienmayonline24h.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1800000', 'website': 'dientu365.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1800000', 'website': 'dienmay88hn.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1800000', 'website': 'dienmayplus.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1800000', 'website': 'dienmaydaian.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1810000', 'website': 'dienmaydaiduong.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1810000', 'website': 'dienmaysaovang.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '1810000', 'website': 'dienmaytnt.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '2430000', 'website': 'dienmayquyetchien.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '2430000', 'website': 'tongkhodienmayonline.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '3300000', 'website': 'dienmayonline365.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '3300000', 'website': 'dienmaytinphat.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '3300000', 'website': 'dienmayonlinevn.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '3300000', 'website': 'dienmayphuminh.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '3900000', 'website': 'hahahaha.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5100000', 'website': 'dienmayhanoi.online', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5180000', 'website': 'dienmaylinhvuong.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5180000', 'website': 'dienmaygiarevn.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5280000', 'website': 'dienmay79.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5350000', 'website': 'muahangtaikho.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5455000', 'website': 'sendo.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5490000', 'website': 'dienmaythuanthanh.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5490000', 'website': 'spcmidea.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5590000', 'website': 'kysudienmay.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5599000', 'website': 'nhanhavui.com.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5600000', 'website': 'dienmaysamtech.com.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5600000', 'website': 'dienlanh50.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5649000', 'website': 'dienlanhsieure.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5650000', 'website': 'dienmayphuckhanh.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5650000', 'website': 'dienmaythienphu.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5700000', 'website': 'dienmayduclong.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5700000', 'website': 'dienmayhoanglinh.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5700000', 'website': 'dienmayttg.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5700000', 'website': 'vidic.com.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5750000', 'website': 'dienmayhd.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5780000', 'website': 'manhnguyen.com.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5790000', 'website': 'shopee.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5790000', 'website': 'tiki.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5790000', 'website': 'dienmaynetbuy.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5800000', 'website': 'vuadienmay.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5800000', 'website': 'tongkhodienmaybamien.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5820000', 'website': 'dienmaykalong.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5890000', 'website': 'dnmart.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5900000', 'website': 'shopdieuhoa.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '5950000', 'website': 'dienlanhlepham.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6160000', 'website': 'dieuhoagiagoc.com.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6215000', 'website': 'dieuhoanhapkhau.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6215000', 'website': 'dienmaydieuhoa.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6215000', 'website': 'dieuhoagiare24h.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6215000', 'website': 'muasamtaikho.com.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6259000', 'website': 'dieuhoagiare.net', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6259000', 'website': 'dieuhoacasper.info', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6270000', 'website': 'sanden.com.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6270000', 'website': 'maylanhanhsao.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6270000', 'website': 'thicongmaylanh.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6270000', 'website': 'giadungngocthuy.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6270000', 'website': 'thegioidieuhoa.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6325000', 'website': 'dieuhoataikho.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6325000', 'website': 'dieuhoanoithat.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6325000', 'website': 'dieuhoavietmy.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6400000', 'website': 'smilemart.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6473500', 'website': 'hoannhat.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6490000', 'website': 'maylanhthehemoi.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6545000', 'website': 'dienlanhmaigia.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6589000', 'website': 'dienmayhlp.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6600000', 'website': 'maylanhmiennam.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6655000', 'website': 'dienmayhaidung.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '6897000', 'website': 'songhung.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '7150000', 'website': 'tannhatviet.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '7150000', 'website': 'havietpro.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '7150000', 'website': 'maylanhbluesea.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '7205000', 'website': 'dienmaycadima.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '7260000', 'website': 'diengiadunghoangkim.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '7469000', 'website': 'dienmaysaigondp.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '7667000', 'website': 'dienmaydonghai.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '7679000', 'website': 'hanoicomputer.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '7909000', 'website': 'banbuondienmay.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '8129000', 'website': 'dienmaythanhtrung.com.vn', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '8140000', 'website': 'dienmayvungtau.com', 'VAT': 1}, {'title': 'Điều hòa Casper 9000 BTU 1 chiều Inverter IC-09TL32 gas R-32', 'price': '8690000', 'website': 'dienmaydungtrang.vn', 'VAT': 1}]
print(len(a))

85
